# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.040632     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
example_weight_summary = example_model.coefficients
print(example_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |   87910.0724923957  | 7873.338143401634  |
| sqft_living |  None |  315.40344055210005 | 3.4557003258547296 |
|   bedrooms  |  None | -65080.215552827525 | 2717.4568544207045 |
|  bathrooms  |  None |  6944.020192638836  | 3923.114931441481  |
+-------------+-------+---------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0]) # should be 271789.505878

271789.5058780301


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_hat = model.predict(data)
    # Then compute the residuals/errors
    resids = outcome - y_hat
    # Then square and add them up
    RSS = (resids * resids).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [8]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print(rss_example_train) # should be 2.7376153833e+14

273761538330193.0


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [9]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [10]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [11]:
# create the remaining 3 features in both TEST and TRAIN data
feat = 'bed_bath_rooms'
train_data[feat] = train_data['bedrooms'] * train_data['bathrooms']
test_data[feat] = test_data['bedrooms'] * test_data['bathrooms']

feat = 'log_sqft_living'
train_data[feat] = train_data['sqft_living'].apply(log)
test_data[feat] = test_data['sqft_living'].apply(log)

feat = 'lat_plus_long'
train_data[feat] = train_data['lat'] + train_data['long']
test_data[feat] = test_data['lat'] + test_data['long']


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [12]:
print(test_data['bedrooms_squared'].mean())
print(test_data['bed_bath_rooms'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['lat_plus_long'].mean())

12.446677701584301
7.503901631591394
7.550274679645938
-74.65333497217307


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [13]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [32]:
# Learn the three models: (don't forget to set validation_set = None)
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression().fit(train_data[model_1_features].to_numpy(), train_data['price'].to_numpy())
reg2 = LinearRegression().fit(train_data[model_2_features].to_numpy(), train_data['price'].to_numpy())
reg3 = LinearRegression().fit(train_data[model_3_features].to_numpy(), train_data['price'].to_numpy())


In [33]:
# Examine/extract each model's coefficients
print('reg1: ', reg1.coef_)
print('reg2: ', reg2.coef_)
print('reg3: ', reg3.coef_)

reg1:  [ 3.12258535e+02 -5.95858879e+04  1.57061089e+04  6.58622995e+05
 -3.09365335e+05]
reg2:  [ 3.06609917e+02 -1.13445239e+05 -7.14611659e+04  6.54848601e+05
 -2.94287544e+05  2.55794155e+04]
reg3:  [ 5.29426831e+02  3.45198164e+04  6.70626341e+04  5.34087018e+05
 -4.06750900e+05 -8.57108317e+03 -6.78904368e+03 -5.61841347e+05
  1.27336119e+05]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [37]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
reg1_y_hat = reg1.predict(train_data[model_1_features].to_numpy())
reg2_y_hat = reg2.predict(train_data[model_2_features].to_numpy())
reg3_y_hat = reg3.predict(train_data[model_3_features].to_numpy())

def get_sq_resids(y_hat, y):
    resids = y_hat - y
    RSS = (resids*resids).sum()
    return RSS

reg1_rss = get_sq_resids(reg1_y_hat, train_data['price'])
reg2_rss = get_sq_resids(reg2_y_hat, train_data['price'])
reg3_rss = get_sq_resids(reg3_y_hat, train_data['price'])
print('reg1 rss: ', reg1_rss)
print('reg2 rss: ', reg2_rss)
print('reg3 rss: ', reg3_rss)


reg1 rss:  967884405886592.5
reg2 rss:  958424272493712.4
reg3 rss:  903438924951753.1


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [38]:
# Compute the RSS on TESTING data for each of the three models and record the values:
reg1_y_hat = reg1.predict(test_data[model_1_features].to_numpy())
reg2_y_hat = reg2.predict(test_data[model_2_features].to_numpy())
reg3_y_hat = reg3.predict(test_data[model_3_features].to_numpy())

def get_sq_resids(y_hat, y):
    resids = y_hat - y
    RSS = (resids*resids).sum()
    return RSS

reg1_rss = get_sq_resids(reg1_y_hat, test_data['price'])
reg2_rss = get_sq_resids(reg2_y_hat, test_data['price'])
reg3_rss = get_sq_resids(reg3_y_hat, test_data['price'])
print('reg1 rss: ', reg1_rss)
print('reg2 rss: ', reg2_rss)
print('reg3 rss: ', reg3_rss)

reg1 rss:  225502363464156.56
reg2 rss:  223379372775350.06
reg3 rss:  259242423114170.56


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.